In [17]:
import requests
import json

# If You like to test, can put url of data app and any entry app
url = '10.168.20.20:5010'
entries = ['5b68a0ecea0637002f6a4e34', '5b97f3583cdad83c6c25aa56']
rolesa = '5b688de7ea0637002f6a4e30'

#url = 'localhost:5010'
#entries = ['5b340835b3bad96ea3a82eff']
#rolesa = '593f49be42d3ed97390f25a7'

In [18]:
pipeline = [
    {'$match': {'_id': {'$in': entries}, 'roles._id': rolesa}},
    {
        '$graphLookup': {
            'from': 'applications',
            'startWith': '$deps._id',
            'connectFromField': 'deps._id',
            'connectToField': '_id',
            'as': 'nodes',
            'maxDepth': 10,
            'depthField': 'steps'
        }
    },
    {'$project': {
                    'name': 1, 'family': 1, 
                    'servers': 1,
                    'datacenters': 1,
                    'deps._id': 1, 'deps.endpoint': 1, 
                    'nodes._id': 1, 'nodes.deps': 1, 
                    'nodes.steps': 1, 'nodes.name': 1, 
                    'nodes.family': 1, 'nodes.servers': 1,
                    'nodes.provider': 1, 'nodes.datacenters': 1
                 }
    }
];

jpipeline = json.dumps(pipeline)

data = requests.post('http://%s/aggregate' % url, json={'entity': 'applications', 'pipeline': jpipeline})
data = data.json()

In [19]:
import sys
import os
import numpy as np
cwd = os.getcwd()
spltted = cwd.split('/')[:-1]
sys.path.append('/'.join(spltted))

import networkx as nx
from app.services.gridOrchestrator import GridOrchestrator
from app.libs.network.baseNetwork import BaseNetwork
from app.libs.histograms.histogram import Histogram
from app.libs.histograms.grid import GridHistogram
from app.libs.drawing.areaSVG import DrawArea
from app.libs.drawing.layoutSVG import DrawLayout

from app.libs.transformDict import append_servers, transform_dict

from svgwrite import Drawing

network = BaseNetwork()
network.make(data.get('items')).get_graph()

In [20]:
class DrawTemplateSVG(object):
    def __init__(self, tmax, servers, grid, darea=DrawArea):
        pass
        
    def draw_app(self, item):
        pass
    

    def add(self, symbol):
        self.dwg.add(symbol)

    def save(self):
        self.dwg.save()
        return self.dwg.tostring()

    def draw_connect(self, node1, node2, details={}):
        pass

In [21]:
from IPython.display import SVG, display
from functools import reduce
from app.repository.externalMaestroData import ExternalMaestroData

print("-------------------------")
Orchestration = GridOrchestrator(network.graph)
Orchestration.create(entries)


och = Orchestration.get_mapping()

servers_id = reduce(append_servers, och[1].values(), [])
servers_id = list(set(servers_id)) #remove duplicate
query = {"_id": servers_id}
ExternalRequest = ExternalMaestroData(owner_id=rolesa, graph_id="asda")
result = ExternalRequest.get_request(path="servers", query=query)

servers = transform_dict(result)

Layout = DrawLayout(*och, servers, DrawTemplateSVG)

Layout.draw_connections(network.graph.edges(data='endpoint'))
Layout.draw_nodes()

Layout.save()

-------------------------


AttributeError: 'DrawTemplateSVG' object has no attribute 'dwg'